# RPC Log Analysis

## Functionalities
- Plot number of RPCs per second.
- Plot instantaneous latency of RPCs.
- Plot latency distribution of RPCs.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
  ...
  [node-n]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
```
These tarballs contain RPC log files named `calls.log`.

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_2021-10-10-18-35-22"

########## LATENCY
# Max expected value
MAX_LATENCY_IN_S = 1
# Bin size
LATENCY_BIN_IN_MS = 1

## Notebook Setup

In [ ]:
# Import libraries.
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import tarfile
import warnings
warnings.filterwarnings("ignore")

# Constants
COMPONENT_TARBALL_PATTERNS = [
    r"^apigateway.*\.tar\.gz$",
    r"^.+_service.*\.tar\.gz$",
]
RPC_LOG_PATTERN = r"^\[(.+)\] pid=(.+) tid=(.+) request_id=(.+) server=(.+) function=(.+) latency=(.+)$"

## Log Parsing

In [ ]:
# Parse logs
rpc = {"node_name": [], "timestamp": [], "request_id": [], "server": [], "function": [], "latency": []}
node_names = os.listdir(os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs"))
for node_name in node_names:
  for tarball_name in os.listdir(os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs", node_name)):
    if sum([1 if re.match(tarball_pattern, tarball_name) else 0 for tarball_pattern in COMPONENT_TARBALL_PATTERNS]):
      tarball_path = os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs", node_name, tarball_name)
      with tarfile.open(tarball_path, "r:gz") as tar:
        for filename in tar.getnames():
          if filename.endswith("calls.log"):
            with tar.extractfile(filename) as log_file:
              for row in log_file:
                rpc_log_match = re.match(RPC_LOG_PATTERN, row.decode("utf-8"))
                if rpc_log_match:
                  timestamp, pid, tid, request_id, server, function, latency = rpc_log_match.groups()
                  rpc["node_name"].append(node_name)
                  rpc["timestamp"].append(datetime.datetime.strptime(timestamp[:-3], "%H:%M:%S.%f"))
                  rpc["request_id"].append(request_id)
                  rpc["server"].append(server)
                  rpc["function"].append(function)
                  rpc["latency"].append(float(latency) * 1000)

In [ ]:
# Build data frame
rpc = pd.DataFrame.from_dict(rpc)

# Get values
function_names = sorted(rpc["function"].unique())
min_timestamp = rpc["timestamp"].values.min()

# (Re) Build columns
rpc["timestamp"] = rpc.apply(lambda r: (r["timestamp"] - min_timestamp).total_seconds(), axis=1)
rpc["window"] = rpc.apply(lambda r: int(r["timestamp"]), axis=1)

## Number of RPCs per Second

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot
fig = plt.figure(figsize=(24, len(function_names) * 12))
for (i, function) in enumerate(function_names):
    df = rpc[(rpc["function"] == function)]
    if MIN_TIME:
        df = df[(df["timestamp"] >= MIN_TIME)]
    if MAX_TIME:
        df = df[(df["timestamp"] <= MAX_TIME)]
    df = df.groupby(["window"])["window"].count()
    df = df.reindex(range(int(df.index.min()), int(df.index.max()) + 1), fill_value=0)
    ax = fig.add_subplot(len(function_names), 1, i + 1)
    ax.grid(alpha=0.75)
    ax.set_xlim((MIN_TIME or 0, MAX_TIME or int(df.index.max())))
    ax.set_ylim((0, int(df.values.max())))
    df.plot(ax=ax, kind="line", title="RPCs per second - %s" % function, xlabel="Time (seconds)",
        ylabel="Requests (count)", color="blue", grid=True)
    plt.subplots_adjust(hspace=0.25)

## Instantaneous Latency of RPCs

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot
fig = plt.figure(figsize=(24, len(function_names) * 12))
for (i, function) in enumerate(function_names):
    df = rpc[(rpc["function"] == function)]
    if MIN_TIME:
        df = df[(df["timestamp"] >= MIN_TIME)]
    if MAX_TIME:
        df = df[(df["timestamp"] <= MAX_TIME)]
    df.set_index("timestamp", inplace=True)
    df.sort_index(inplace=True)
    ax = fig.add_subplot(len(function_names), 1, i + 1)
    ax.grid(alpha=0.75)
    ax.set_xlim((MIN_TIME or 0, MAX_TIME or int(df.index.max())))
    ax.set_ylim((0, df["latency"].max()))
    df["latency"].plot(ax=ax, kind="line", title="Instantaneous Latency - %s" % function, xlabel="Time (seconds)",
        ylabel="Latency (milliseconds)", color="purple", grid=True)
    plt.subplots_adjust(hspace=0.25)

## Latency Distribution of RPCs

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot
fig = plt.figure(figsize=(24, len(function_names) * 12))
for (i, function) in enumerate(function_names):
    df = rpc[(rpc["function"] == function)]
    if MIN_TIME:
        df = df[(df["timestamp"] >= MIN_TIME)]
    if MAX_TIME:
        df = df[(df["timestamp"] <= MAX_TIME)]
    if not len(df):
        continue
    df["latency_bin"] = df.apply(lambda r: int(r["latency"] // LATENCY_BIN_IN_MS), axis=1)
    ax = fig.add_subplot(len(function_names), 1, i + 1)
    ax.set_yscale("log")
    ax.grid(alpha=0.75)
    ax.set_xlim((0, (1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S))
    df["latency_bin"].plot(ax=ax, kind="hist", title="Latency Distribution - %s" % function,
        xlabel="Latency (milliseconds)", ylabel="Requests (count)",
        bins=range((1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S), grid=True)
    plt.subplots_adjust(hspace=0.25)

## Statistics

In [ ]:
for (i, function) in enumerate(function_names):
    df = rpc[(rpc["function"] == function)]
    print(function)
    print("  Number of RPCs/s")
    print("    Total:       %7d" % df.shape[0])
    print("    Avg:         %7.2f" % (df.shape[0] / (df["timestamp"].max() - df["timestamp"].min())))
    print("  Latency (ms)")
    print("  P99.9:         %7.2f" % (df["latency"].quantile(0.999)))
    print("    P99:         %7.2f" % (df["latency"].quantile(0.99)))
    print("    P95:         %7.2f" % (df["latency"].quantile(0.95)))
    print("    P50:         %7.2f" % (df["latency"].quantile(0.50)))
    print("    Avg:         %7.2f" % (df["latency"].mean()))
    print("    Std:         %7.2f" % (df["latency"].std()))